In [2]:
import pandas as pd
import numpy as np

NOTES:

    - include adm_stat = 11, 12, 14
    - when running this program on a new FY for the first time, you have to update the FYs list in the CR_GR_TR functions

In [3]:
##retrieve data
pers_dirty = pd.read_csv('pers_dtl2.csv')
pers_dirty2 = pers_dirty.loc[pers_dirty['RACE_HISPANIC']==1,:]
pers_dirty2

,TECH_ID,COHORT_ENTERING_YEAR_TERM_CODE,FY,Semester,ADM_STAT,RACE_HISPANIC,FIRST_FALL_RETAINED,FIRST_FALL_TRANSFERRED,FIRST_FALL_GRADUATED,FIRST_FALL_TRANS_GRADUATED,...,EIGTH_FALL_RETAINED,EIGTH_FALL_TRANSFERRED,EIGTH_FALL_GRADUATED,EIGTH_FALL_TRANS_GRADUATED,EIGTH_FALL_SUCCESS,EIGTH_SPRING_RETAINED,EIGTH_SPRING_TRANSFERRED,EIGTH_SPRING_GRADUATED,EIGTH_SPRING_TRANS_GRADUATED,EIGTH_SPRING_SUCCESS
24,5399,20053,2005,3,11,1,1,0,0,0,...,0,0,0,1,1,0,0,0,1,1
25,5677,20103,2010,3,11,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,5682,20103,2010,3,11,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,5758,20085,2008,5,11,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32,6420,20033,2003,3,12,1,1,0,0,0,...,0,0,1,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22124,16501143,20243,2024,3,11,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22137,16508961,20243,2024,3,11,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22141,16511315,20243,2024,3,11,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22149,16515381,20243,2024,3,14,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#rates for graduation and transfer for cohorts with no time cutoff
def CR_GR_TR_AT(df):
    compRates = pd.DataFrame(columns=['FY','Term','Cohort_size','nTransferred','nGraduated','CR','GR','TR'])
    FYs = [2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
    Sems = [3,5]
    for sem in Sems:
        for fy in FYs:
            df_new = df.loc[(df['FY']==fy) & (df['Semester']==sem),:]
            cohort_size = 0
            transferred_wi3 = 0
            graduated_wi3 = 0
            df_new.reset_index()
            df_new['Transferred_within3'] = df_new['FIRST_FALL_TRANSFERRED'] + df_new['FIRST_SPRING_TRANSFERRED'] + df_new['SECOND_FALL_TRANSFERRED'] + df_new['SECOND_SPRING_TRANSFERRED'] + df_new['THIRD_FALL_TRANSFERRED'] + df_new['THIRD_SPRING_TRANSFERRED'] + df_new['FOURTH_FALL_TRANSFERRED'] + df_new['FOURTH_SPRING_TRANSFERRED'] + df_new['FIFTH_FALL_TRANSFERRED'] + df_new['FIFTH_SPRING_TRANSFERRED'] + df_new['SIXTH_FALL_TRANSFERRED'] + df_new['SIXTH_SPRING_TRANSFERRED'] + df_new['SEVENTH_FALL_TRANSFERRED'] + df_new['SEVENTH_SPRING_TRANSFERRED'] + df_new['EIGTH_FALL_TRANSFERRED'] + df_new['EIGTH_SPRING_TRANSFERRED']
            df_new['Graduated_within3'] = df_new['FIRST_FALL_GRADUATED'] + df_new['FIRST_SPRING_GRADUATED'] + df_new['SECOND_FALL_GRADUATED'] + df_new['SECOND_SPRING_GRADUATED'] + df_new['THIRD_FALL_GRADUATED'] + df_new['THIRD_SPRING_GRADUATED'] + df_new['FOURTH_FALL_GRADUATED'] + df_new['FOURTH_SPRING_GRADUATED'] + df_new['FIFTH_FALL_GRADUATED'] + df_new['FIFTH_SPRING_GRADUATED'] + df_new['SIXTH_FALL_GRADUATED'] + df_new['SIXTH_SPRING_GRADUATED'] + df_new['SEVENTH_FALL_GRADUATED'] + df_new['SEVENTH_SPRING_GRADUATED'] + df_new['EIGTH_FALL_GRADUATED'] + df_new['EIGTH_SPRING_GRADUATED']
            for i in df_new.index:
                cohort_size = cohort_size + 1
                if df_new.loc[i,'Transferred_within3'] > 0:
                    transferred_wi3 = transferred_wi3 + 1
                if df_new.loc[i,'Graduated_within3'] > 0:
                    graduated_wi3 = graduated_wi3 + 1
                else:
                    continue
            CR = (transferred_wi3+graduated_wi3)/cohort_size *100
            GR = (graduated_wi3)/cohort_size *100
            TR = (transferred_wi3)/cohort_size *100
            sdf = pd.DataFrame({'FY':fy,'Term':sem,'Cohort_size':cohort_size,'nTransferred':transferred_wi3,'nGraduated': graduated_wi3,'CR':CR,'GR':GR,'TR':TR}, index=[1])
            compRates = pd.concat([compRates,sdf])
    return compRates

#rates for graduation and transfer within 3 yrs of starting
def CR_GR_TR_3yr(df):
    compRates = pd.DataFrame(columns=['FY','Term','Cohort_size','nTransferred','nGraduated','CR','GR','TR'])
    FYs = [2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023]
    Sems = [3,5]
    for sem in Sems:
        for fy in FYs:
            df_new = df.loc[(df['FY']==fy) & (df['Semester']==sem),:]
            cohort_size = 0
            transferred_wi3 = 0
            graduated_wi3 = 0
            df_new.reset_index()
            df_new['Transferred_within3'] = df_new['FIRST_FALL_TRANSFERRED'] + df_new['FIRST_SPRING_TRANSFERRED'] + df_new['SECOND_FALL_TRANSFERRED'] + df_new['SECOND_SPRING_TRANSFERRED'] + df_new['THIRD_FALL_TRANSFERRED'] + df_new['THIRD_SPRING_TRANSFERRED']
            df_new['Graduated_within3'] = df_new['FIRST_FALL_GRADUATED'] + df_new['FIRST_SPRING_GRADUATED'] + df_new['SECOND_FALL_GRADUATED'] + df_new['SECOND_SPRING_GRADUATED'] + df_new['THIRD_FALL_GRADUATED'] + df_new['THIRD_SPRING_GRADUATED']
            for i in df_new.index:
                cohort_size = cohort_size + 1
                if df_new.loc[i,'Transferred_within3'] > 0:
                    transferred_wi3 = transferred_wi3 + 1
                if df_new.loc[i,'Graduated_within3'] > 0:
                    graduated_wi3 = graduated_wi3 + 1
                else:
                    continue
            CR = (transferred_wi3+graduated_wi3)/cohort_size *100
            GR = (graduated_wi3)/cohort_size *100
            TR = (transferred_wi3)/cohort_size *100
            sdf = pd.DataFrame({'FY':fy,'Term':sem,'Cohort_size':cohort_size,'nTransferred':transferred_wi3,'nGraduated': graduated_wi3,'CR':CR,'GR':GR,'TR':TR}, index=[1])
            compRates = pd.concat([compRates,sdf])
    return compRates
    
#this one is if you want to run an individual year and semester   
def rates(df, fy, sem):
    #uses the year term to first select the appropriate data from the df
    df_new = df.loc[(df['FY']==fy) & (df['Semester']==sem),:]
    #establish your counters
    cohort_size = 0
    transferred_wi3 = 0
    graduated_wi3 = 0
    df_new.reset_index()
    df_new['Transferred_within3'] = df_new['FIRST_FALL_TRANSFERRED'] + df_new['FIRST_SPRING_TRANSFERRED'] + df_new['SECOND_FALL_TRANSFERRED'] + df_new['SECOND_SPRING_TRANSFERRED'] + df_new['THIRD_FALL_TRANSFERRED'] + df_new['THIRD_SPRING_TRANSFERRED']
    df_new['Graduated_within3'] = df_new['FIRST_FALL_GRADUATED'] + df_new['FIRST_SPRING_GRADUATED'] + df_new['SECOND_FALL_GRADUATED'] + df_new['SECOND_SPRING_GRADUATED'] + df_new['THIRD_FALL_GRADUATED'] + df_new['THIRD_SPRING_GRADUATED']
    for i in df_new.index:
        cohort_size = cohort_size + 1
        if df_new.loc[i,'Transferred_within3'] > 0:
           transferred_wi3 = transferred_wi3 + 1
        if df_new.loc[i,'Graduated_within3'] > 0:
           graduated_wi3 = graduated_wi3 + 1
        else:
            continue
    CR = (transferred_wi3+graduated_wi3)/cohort_size *100
    GR = (graduated_wi3)/cohort_size *100
    TR = (transferred_wi3)/cohort_size *100
    print(cohort_size, transferred_wi3, graduated_wi3, CR, GR, TR)

In [6]:
compRates_3yr = CR_GR_TR_3yr(pers_dirty2)
compRates_AT = CR_GR_TR_AT(pers_dirty2)

C:\Users\qf3636ll\AppData\Local\Temp\ipykernel_17664\4241578045.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Transferred_within3'] = df_new['FIRST_FALL_TRANSFERRED'] + df_new['FIRST_SPRING_TRANSFERRED'] + df_new['SECOND_FALL_TRANSFERRED'] + df_new['SECOND_SPRING_TRANSFERRED'] + df_new['THIRD_FALL_TRANSFERRED'] + df_new['THIRD_SPRING_TRANSFERRED']
C:\Users\qf3636ll\AppData\Local\Temp\ipykernel_17664\4241578045.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['Graduated_within3'] = 

In [ ]:
#compRates_AT
#compRates_3yr

In [7]:
compRates_3yr.to_csv('compRates_3yr.csv', index=False)
compRates_AT.to_csv('compRates_AT.csv', index=False)